# $sa$-rectangular RMDPでの方策勾配について

* [First-order Policy Optimization for Robust Markov Decision Process](https://arxiv.org/abs/2209.10579)
    * Remark 1.1でなぜ遷移のUncertainty setを考えるのか？みたいな話をしていて良い

MDP$\mathcal{M}_u$での価値を次で定義します：
$$
V_u^\pi(s)=\mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t c\left(s_t, a_t\right) \mid s_0=s, a_t \sim \pi\left(\cdot \mid s_t\right), s_{t+1} \sim \mathbb{P}_u\left(\cdot \mid s_t, a_t\right)\right], \quad \forall s \in \mathcal{S} .
$$

このとき，ロバストMDPでの最適方策は次で定義されます：
$$
\min \left\{V_r^\pi(s):=\max _{u \in \mathcal{U}} V_u^\pi(s): \pi \in \Pi\right\}
$$

## ロバストMDPが重要な例

ロバストMDPはいつ重要になるでしょうか？次のようなケースを考えてみましょう：

![Robust-MDP](figs/SA-Robust-MDP.png)

**左のMDP**

* 状態$S_0$では２つの行動$\{L, R\}$があります．$R$だとコスト$-1$，$L$だとコスト$0$が発生します．
* また，状態$S_{k-1, L}$から$S_{k, L}$の部分でコスト$-(1+\epsilon) \gamma^{-k+1}$が発生します．ここで$\epsilon <<1$です．例えば$\epsilon=0.01$など．
    * つまり，左に行くと価値$-(1+\epsilon)$，右に行くと価値$-1$が発生します．よって左に行くのが最適解です．

**右のMDP**

* $\mathbb{P}_u\left(S_{m+1, L} \mid S_{m, L}\right)=p, \mathbb{P}_u\left(S_{m, R} \mid S_{m, L}\right)=1-p$とします．
* $S_{i, R}$に到達するとずっとその場に留まります．
* このとき，左に行くと価値は$-(1+\epsilon) p^{k-1}$であり，これは$k\to \infty$で価値が$0$になります．よって，このMDPでは右に行くのが最適解です．

左右のMDPでは遷移が少し違うだけですが，その最適方策と価値が全然違います．
このような環境では左右等確率で選択するのは良い方策にはならず，良さそうな解は右を選択する方策です．
右を選択する方策は$\mathcal{M}$では$\epsilon$-最適方策，そして$\mathcal{M}_u$では最適方策になります．

## SA-rectangular set

以降では，Uncertainty setが$(s, a)$-rectangularであるとします．

$$
\mathbb{P}_u(\cdot \mid s, a)=\mathbb{P}_{\mathbb{N}}(\cdot \mid s, a)+u(\cdot \mid s, a), \forall(s, a) \in \mathcal{S} \times \mathcal{A},
$$

ここで，$\mathbb{P}_N(\cdot |s, a)$はノミナルモデル，$u \in \mathbb{R}^{|\mathcal{S}| \times|\mathcal{S}| \times|\mathcal{A}|}$はノイズです．
$\mathbb{P}_N$をもつMDPをノミナルMDPと呼ぶことにします．
$\mathcal{U}$が次を満たすとき，このUncertainty setを$sa$-rectangularと言います：

$$
\mathcal{U}=\Pi_{s \in \mathcal{S}, a \in \mathcal{A}} \mathcal{U}_{s, a} \text {, where } \mathcal{U}_{s, a}=\{u(\cdot \mid s, a): u \in \mathcal{U}\} \text {. }
$$

以降，$\mathcal{U}$はコンパクトであるとします．
コンパクトな$\mathcal{U}$については次のBellman方程式が成立します：

$$
V_r^\pi(s)=\sum_{a \in \mathcal{A}} c(s, a) \pi(a \mid s)+\gamma \sum_{a \in \mathcal{A}} \pi(a \mid s) \max _{u \in \mathcal{U}} \sum_{s^{\prime} \in \mathcal{S}} \mathbb{P}_u\left(s^{\prime} \mid s, a\right) V_r^\pi\left(s^{\prime}\right), \quad \forall s \in \mathcal{S} .
$$

$$
Q_r^\pi(s, a)=c(s, a)+\max _{u \in \mathcal{U}} \gamma \sum_{s^{\prime} \in \mathcal{S}} \mathbb{P}_u\left(s^{\prime} \mid s, a\right) \sum_{a^{\prime} \in \mathcal{A}} \pi\left(a^{\prime} \mid s^{\prime}\right) Q_r^\pi\left(s^{\prime}, a^{\prime}\right), \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A} .
$$


最後に，最悪ケースの遷移モデルを次で定義します：
$$
u_\pi(\cdot \mid s, a) \in \underset{u(\mid s, a) \in \mathcal{U}_{s, a}}{\operatorname{Argmax}} \sum_{s^{\prime} \in \mathcal{S}} \mathbb{P}_u\left(s^{\prime} \mid s, a\right) V_r^\pi\left(s^{\prime}\right), \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A},
$$


## ロバストMDPと勾配

次の目的関数を通じてロバストMDPを解くことを考えましょう．
$$
\min _{\pi \in \Pi}\left\{f_\rho(\pi):=\mathbb{E}_{s \sim \rho} V_r^\pi(s)\right\},
$$

この$f_\rho$に対して，次のように勾配を定義します：

---

任意の関数$f : \Pi \to \mathbb{R}$について，方策勾配$\nabla f(\pi)$を次で定義します：
$$
\lim _{\delta \rightarrow 0, \pi+\delta \in \Pi}|f(\pi+\delta)-f(\pi)-\langle\nabla f(\pi), \delta\rangle| /\|\delta\|_2 \rightarrow 0 .
$$

---

よく知られているように，方策勾配は
$$
\nabla V_u^\pi(s)\left[s^{\prime}, a\right]=\frac{1}{1-\gamma} d_s^{\pi, u}\left(s^{\prime}\right) Q_u^\pi\left(s^{\prime}, a\right), \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A},
$$
になります．

一方で，ロバストMDPにおけるフレシェ劣勾配は次で得られます：

---

$\bar{f}_\rho: \mathbb{R}^{|\mathcal{S}||\mathcal{A}|} \rightarrow \bar{R}=\mathbb{R} \cup\{+\infty\}$
を$f_\rho$の拡張関数とします．つまり，
$\bar{f}_\rho(\pi)=f_\rho(\pi)$ for any $\pi \in \Pi$ and $\bar{f}_\rho(\pi)=\infty$です．

ここで，
$$
\nabla f_\rho(\pi)[s, a]=\frac{1}{1-\gamma} d_\rho^{\pi, u_\pi}(s) Q_{u_\pi}^\pi(s, a), \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A}
$$
を定義しましょう．このとき，$\nabla f_\rho(\pi)$は$\bar{f}_\rho$の$\pi$でのフレシェ劣微分です．

**証明**

$\pi, \pi' \in \Pi$について，$\delta=\pi'-\pi$とします．
このとき，
$$
\begin{aligned}
 V_{u_{\pi^{\prime}}}^{\pi^{\prime}}(s)-V_{u_\pi}^\pi(s)&=V_{u_{\pi^{\prime}}}^{\pi^{\prime}}(s)-V_{u_\pi}^{\pi^{\prime}}(s)+V_{u_\pi}^{\pi^{\prime}}(s)-V_{u_\pi}^\pi(s)\\
& \stackrel{(a)}{\geq} V_{u_\pi}^{\pi^{\prime}}(s)-V_{u_\pi}^\pi(s) \\
& \stackrel{(b)}{=} \frac{1}{1-\gamma} \sum_{s^{\prime} \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_s^{\pi, u_\pi}\left(s^{\prime}\right) Q_{u_\pi}^\pi\left(s^{\prime}, a\right) \delta\left(a \mid s^{\prime}\right)+\mathcal{O}\left(\left\|\pi-\pi^{\prime}\right\|_2\right),
\end{aligned}
$$

が成立します．ここで，(a)は$u_\pi$の定義より成立，(b)は多分Performance difference lemma的なことをしたら成立します（論文参照）．
よって，$f_\rho$の定義より，

$$
\begin{aligned}
f_\rho\left(\pi^{\prime}\right)-f_\rho(\pi) & \geq \frac{1}{1-\gamma} \sum_{s \in \mathcal{S}} \rho(s) \sum_{s^{\prime} \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_s^{\pi, u_\pi}\left(s^{\prime}\right) Q_{u_\pi}^\pi\left(s^{\prime}, a\right) \delta\left(a \mid s^{\prime}\right)+\mathcal{O}\left(\left\|\pi-\pi^{\prime}\right\|_2\right) \\
& =\frac{1}{1-\gamma} \sum_{s \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_\rho^{\pi, u_\pi}(s) Q_{u_\pi}^\pi(s, a) \delta(a \mid s)+\mathcal{O}\left(\left\|\pi-\pi^{\prime}\right\|_2\right),
\end{aligned}
$$

これについて，

$$
\liminf _{\pi^{\prime} \rightarrow \pi, \pi^{\prime} \neq \pi}\left(f_\rho\left(\pi^{\prime}\right)-f_\rho(\pi)-\frac{1}{1-\gamma} \sum_{s \in \mathcal{S}} \sum_{a \in \mathcal{A}} d_\rho^{\pi, u_\pi}(s) Q_{u_\pi}^\pi(s, a) \delta(a \mid s)\right) /\left\|\pi^{\prime}-\pi\right\|_2 \geq 0
$$

が成立するので，ちゃんとフレシェ劣微分になっています．

---

この勾配について勾配降下法を考えるのも良いですが，停留点の最適性について考えなければいけず，ちょっとめんどくさいです．
そこで，次の変分不等式を利用したアプローチについて考えてみましょう．

## 変分不等式を使った定式化

まず，次が成立します：

$$
V_r^{\pi^{\prime}}(s)-V_r^\pi(s) \leq \frac{1}{1-\gamma} \mathbb{E}_{s^{\prime} \sim d_s^{\pi^{\prime}, u_{\pi^{\prime}}}}\left\langle Q_r^\pi, \pi^{\prime}-\pi\right\rangle_{s^{\prime}} .
$$

証明は省略．これより，
$$
0 \leq V_r^\pi(s)-V_r^{\pi^*}(s) \leq \frac{1}{1-\gamma} \mathbb{E}_{s^{\prime} \sim d_s^{\pi, u \pi}}\left\langle Q_r^{\pi^*}, \pi-\pi^*\right\rangle_s=\left\langle\mathcal{F}_s\left(\pi, \pi^*\right), \pi-\pi^*\right\rangle,
$$
が成立します．ここで，$\mathcal{F}_s\left(\pi, \pi^{\prime}\right)\left[s^{\prime}, a\right]=\frac{1}{1-\gamma} d_s^{\pi, u_\pi}\left(s^{\prime}\right) Q_r^{\pi^{\prime}}\left(s^{\prime}, a\right) \quad \forall\left(s^{\prime}, a\right) \in \mathcal{S} \times \mathcal{A}$としました．

$\pi^*$の最適性より，$\left\langle\mathcal{F}\left(\pi, \pi^*\right), \pi-\pi^*\right\rangle \geq 0$が任意の$\pi \in \Pi$で成立します．ここで，$\pi_\alpha=\alpha \pi+(1-\alpha) \pi^*$とすると，

$$
\left\langle\mathcal{F}_s\left(\pi_\alpha, \pi^*\right), \pi_\alpha-\pi^*\right\rangle=\alpha\left\langle\mathcal{F}_s\left(\pi_\alpha, \pi^*\right), \pi-\pi^*\right\rangle \geq 0
$$

なので，

$$
\left\langle\mathcal{F}_s\left(\pi_\alpha, \pi^*\right), \pi-\pi^*\right\rangle \geq 0
$$

です．ごにょごにょすると（TODO: ），
$$
\begin{aligned}
\lim _{\alpha \downarrow 0}\left\langle\mathcal{F}_s\left(\pi_\alpha, \pi^*\right), \pi-\pi^*\right\rangle \geq 0 & \Rightarrow\left\langle\mathcal{G}_s\left(\pi^*\right), \pi-\pi^*\right\rangle \geq 0 \\
\mathcal{G}_s\left(\pi^*\right)\left[s^{\prime}, a\right] & =\frac{1}{1-\gamma} d_s^{\pi^*, u_{\pi^*}}\left(s^{\prime}\right) Q_r^{\pi^*}\left(s^{\prime}, a\right), \quad \forall\left(s^{\prime}, a\right) \in \mathcal{S} \times \mathcal{A}
\end{aligned}
$$
であることがわかります．よって，最適方策は次の変分不等式を解けば得られるわけですね：

$$
\left\langle\mathbb{E}_{s \sim \rho}\left[\mathcal{G}_s\left(\pi^*\right)\right], \pi-\pi^*\right\rangle \geq 0, \quad \forall \pi \in \Pi .
$$

ちなみにロバストではないMDPの最適方策は
$$
\begin{aligned}
&\left\langle\mathbb{E}_{s \sim \nu^*}\left[\mathcal{G}_s^N\left(\pi^*\right)\right], \pi-\pi^*\right\rangle \geq 0,\\
&\mathcal{G}_s^N\left(\pi^*\right)\left[s^{\prime}, a\right]=\frac{1}{1-\gamma} d_s^{\pi^*}\left(s^{\prime}\right) Q^{\pi^*}\left(s^{\prime}, a\right), \quad \forall\left(s^{\prime}, a\right) \in \mathcal{S} \times \mathcal{A},
\end{aligned}
$$

で求まります（TODO: [Policy Mirror Descent for Reinforcement Learning: Linear Convergence, New Sampling Complexity, and Generalized Problem Classes](https://arxiv.org/abs/2102.00135)を読め）．ここで$\nu^*$は最適方策の定常状態分布です．つまり，Q関数をロバストにしただけですね．

通常のMDPの場合はPerformance difference lemmaを利用すると
$$
\left\langle\mathbb{E}_{s \sim \nu^*}\left[\mathcal{G}_s^N(\pi)\right], \pi-\pi^*\right\rangle=\mathbb{E}_{s \sim \nu^*}\left[V^\pi(s)-V^{\pi^*}(s)\right]=f_{\nu^*}(\pi)-f_{\nu^*}\left(\pi^*\right)>0,
$$
が成立し，これを利用して変分不等式を解くことができます．しかし，実はロバストMDPではこれが成立しません（Proposition 2.3参照）．


